In [1]:
import sys, os
sys.path.insert(0,"../code")

import pandas as pd
import numpy as np
import geopandas as gpd
import pyproj

%load_ext autoreload
%autoreload 2

# Unzip data and load into DataFrame 

In [2]:
import zipfile

def load_data(path_data="../data", link="https://www.kaggle.com/c/6960/download-all"): 
    
    if not os.path.exists(path_data): 
        os.mkdir(path_data)
    
    if not os.path.exists(os.path.join(path_data,"nyc-taxi-trip-duration")):
        
        if not os.path.exists(os.path.join(path_data,"nyc-taxi-trip-duration.zip")):
            print(f"Please download data from {link} into {path_data}.")
        else: 
            zip_obj = zipfile.ZipFile(os.path.join(path_data,"nyc-taxi-trip-duration.zip"), 'r')
            zip_obj.extractall(os.path.join(path_data,"nyc-taxi-trip-duration"))
            zip_obj.close()
            zip_obj = zipfile.ZipFile(os.path.join(path_data,"nyc-taxi-trip-duration",'train.zip'), 'r')
            zip_obj.extractall(os.path.join(path_data,"nyc-taxi-trip-duration"))
            zip_obj.close()

In [3]:
load_data()

In [4]:
PATH_DATA = os.path.join("..\data","nyc-taxi-trip-duration","train","train.csv")

df_rides = pd.read_csv(PATH_DATA,index_col=0,
                       parse_dates=[2,3],
                       dtype={'store_and_fwd_flag':'category','vendor_id':'category','passenger_count':'int8',})

In [5]:
## Split data into preliminary test and train in order to get an estimate without commiting to Kaggel 
df_train_sample = df_rides.sample(frac=0.15)
df_rides = df_rides.loc[set(df_rides.index)-set(df_train_sample.index)]
print(df_train_sample.shape)
print(df_rides.shape)
df_train_sample.to_pickle(os.path.join("../data","df_train_sample.pickle"))

(218797, 10)
(1239847, 10)


##  Basic data exploration 

In [6]:
print(df_rides.info())
df_rides.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1239847 entries, id0161319 to id3745725
Data columns (total 10 columns):
vendor_id             1239847 non-null category
pickup_datetime       1239847 non-null datetime64[ns]
dropoff_datetime      1239847 non-null datetime64[ns]
passenger_count       1239847 non-null int8
pickup_longitude      1239847 non-null float64
pickup_latitude       1239847 non-null float64
dropoff_longitude     1239847 non-null float64
dropoff_latitude      1239847 non-null float64
store_and_fwd_flag    1239847 non-null category
trip_duration         1239847 non-null int64
dtypes: category(2), datetime64[ns](2), float64(4), int64(1), int8(1)
memory usage: 79.2+ MB
None


,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
id,,,,,,,,,,
id0161319,2,2016-01-19 20:31:29,2016-01-19 20:39:14,1,-73.993294,40.729328,-74.008316,40.716763,N,465
id2074074,2,2016-02-03 11:52:45,2016-02-03 12:03:18,2,-73.980499,40.780331,-73.980919,40.765862,N,633
id2653850,2,2016-05-18 20:58:37,2016-05-18 21:04:50,1,-73.980728,40.738369,-73.993721,40.738682,N,373
id0836600,2,2016-05-20 15:59:37,2016-05-20 16:07:38,1,-73.982162,40.774487,-73.982979,40.781868,N,481
id0184401,1,2016-06-05 12:53:33,2016-06-05 13:21:22,1,-73.992538,40.751705,-73.975433,40.717648,N,1669


In [7]:
pd.isna(df_rides).sum()

vendor_id             0
pickup_datetime       0
dropoff_datetime      0
passenger_count       0
pickup_longitude      0
pickup_latitude       0
dropoff_longitude     0
dropoff_latitude      0
store_and_fwd_flag    0
trip_duration         0
dtype: int64

In [8]:
df_rides.describe()

,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration
count,1.239847e+06,1.239847e+06,1.239847e+06,1.239847e+06,1.239847e+06,1.239847e+06
mean,1.664836e+00,-7.397348e+01,4.075090e+01,-7.397342e+01,4.075179e+01,9.601589e+02
std,1.314792e+00,7.496616e-02,3.359400e-02,7.460873e-02,3.636141e-02,5.517635e+03
min,0.000000e+00,-1.219333e+02,3.435970e+01,-1.219333e+02,3.218114e+01,1.000000e+00
25%,1.000000e+00,-7.399187e+01,4.073732e+01,-7.399133e+01,4.073589e+01,3.970000e+02
50%,1.000000e+00,-7.398175e+01,4.075410e+01,-7.397975e+01,4.075452e+01,6.620000e+02
75%,2.000000e+00,-7.396735e+01,4.076836e+01,-7.396302e+01,4.076979e+01,1.075000e+03
max,8.000000e+00,-6.133553e+01,5.188108e+01,-6.133553e+01,4.392103e+01,3.526282e+06


In [9]:
assert(df_rides.index.unique().shape[0]==df_rides.shape[0])
assert(np.all(df_rides.dropoff_datetime>df_rides.pickup_datetime))
assert(np.all(df_rides.passenger_count>=0))
#assert(np.allclose((df_rides.dropoff_datetime-df_rides.pickup_datetime).dt.seconds.values,df_rides.trip_duration,))

## Preprocessing of data

### Geographic data

In [10]:
from shapely.geometry import Point

def transform_to_geodf(df_rides,set_geometry_col='pickup'): 
        df_rides['pickup_geom'] = list(zip(df_rides.pickup_longitude,df_rides.pickup_latitude))
        df_rides['pickup_geom'] = df_rides['pickup_geom'].apply(Point)
        df_rides['dropoff_geom'] = list(zip(df_rides.dropoff_longitude,df_rides.dropoff_latitude))
        df_rides['dropoff_geom'] = df_rides['dropoff_geom'].apply(Point)
        df_rides = gpd.GeoDataFrame(df_rides,geometry=f'{set_geometry_col}_geom',crs={"init":"epsg:4326"})      
        return df_rides

In [11]:
df_rides = transform_to_geodf(df_rides)
df_rides.head()

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,pickup_geom,dropoff_geom
id,,,,,,,,,,,,
id0161319,2,2016-01-19 20:31:29,2016-01-19 20:39:14,1,-73.993294,40.729328,-74.008316,40.716763,N,465,POINT (-73.99329376220702 40.72932815551758),POINT (-74.00831604003906 40.71676254272461)
id2074074,2,2016-02-03 11:52:45,2016-02-03 12:03:18,2,-73.980499,40.780331,-73.980919,40.765862,N,633,POINT (-73.98049926757811 40.78033065795898),POINT (-73.98091888427734 40.76586151123047)
id2653850,2,2016-05-18 20:58:37,2016-05-18 21:04:50,1,-73.980728,40.738369,-73.993721,40.738682,N,373,POINT (-73.98072814941406 40.73836898803711),POINT (-73.99372100830078 40.73868179321289)
id0836600,2,2016-05-20 15:59:37,2016-05-20 16:07:38,1,-73.982162,40.774487,-73.982979,40.781868,N,481,POINT (-73.98216247558594 40.77448654174805),POINT (-73.98297882080078 40.78186798095703)
id0184401,1,2016-06-05 12:53:33,2016-06-05 13:21:22,1,-73.992538,40.751705,-73.975433,40.717648,N,1669,POINT (-73.99253845214845 40.75170516967773),POINT (-73.97543334960938 40.71764755249024)


In [12]:
# Correct utm-zone: {'init': 'epsg:32618'}

def convert_projection_to_utm(df,col_x_source,col_y_source,
                              col_x_dest = 'x_utm', col_y_dest = 'y_utm',
                              projection_source=pyproj.Proj("+init=EPSG:4326"),
                              projection_dest=pyproj.Proj("+init=EPSG:32618")): 
    x,y = pyproj.transform(projection_source, projection_dest,df[col_x_source].values,df[col_y_source].values)
    return df.assign(**{col_x_dest:x,col_y_dest:y})

df_rides = convert_projection_to_utm(df_rides,col_x_source='pickup_longitude',col_y_source='pickup_latitude',col_x_dest="pickup_x_utm",col_y_dest='pickup_y_utm')
df_rides = convert_projection_to_utm(df_rides,col_x_source='dropoff_longitude',col_y_source='dropoff_latitude',col_x_dest="dropoff_x_utm",col_y_dest='dropoff_y_utm')
df_rides.head()


,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,pickup_geom,dropoff_geom,pickup_x_utm,pickup_y_utm,dropoff_x_utm,dropoff_y_utm
id,,,,,,,,,,,,,,,,
id0161319,2,2016-01-19 20:31:29,2016-01-19 20:39:14,1,-73.993294,40.729328,-74.008316,40.716763,N,465,POINT (-73.99329376220702 40.72932815551758),POINT (-74.00831604003906 40.71676254272461),585011.603343,4.509198e+06,583758.775727,4.507789e+06
id2074074,2,2016-02-03 11:52:45,2016-02-03 12:03:18,2,-73.980499,40.780331,-73.980919,40.765862,N,633,POINT (-73.98049926757811 40.78033065795898),POINT (-73.98091888427734 40.76586151123047),586026.281871,4.514872e+06,586009.532571,4.513265e+06
id2653850,2,2016-05-18 20:58:37,2016-05-18 21:04:50,1,-73.980728,40.738369,-73.993721,40.738682,N,373,POINT (-73.98072814941406 40.73836898803711),POINT (-73.99372100830078 40.73868179321289),586061.075414,4.510214e+06,584963.622848,4.510236e+06
id0836600,2,2016-05-20 15:59:37,2016-05-20 16:07:38,1,-73.982162,40.774487,-73.982979,40.781868,N,481,POINT (-73.98216247558594 40.77448654174805),POINT (-73.98297882080078 40.78186798095703),585893.464665,4.514222e+06,585815.073138,4.515040e+06
id0184401,1,2016-06-05 12:53:33,2016-06-05 13:21:22,1,-73.992538,40.751705,-73.975433,40.717648,N,1669,POINT (-73.99253845214845 40.75170516967773),POINT (-73.97543334960938 40.71764755249024),585046.878265,4.511683e+06,586534.983744,4.507919e+06


In [13]:
def calc_distance(df,col_x1_utm,col_x2_utm,col_y1_utm,col_y2_utm,type_='beeline'): 
    if type_ =='beeline': 
        distance = np.sqrt((df[col_x1_utm]-df[col_x2_utm])**2+(df[col_y1_utm]-df[col_y2_utm])**2) 
    elif type_ =='manhattan': 
        distance = np.abs((df[col_x1_utm]-df[col_x2_utm]))+np.abs((df[col_y1_utm]-df[col_y2_utm]))
    return df.assign(**{f"distance_{type_}":distance})

df_rides = calc_distance(df_rides,
                         col_x1_utm='pickup_x_utm',col_x2_utm='dropoff_x_utm',
                         col_y1_utm='pickup_y_utm',col_y2_utm="dropoff_y_utm",
                         type_='beeline')

df_rides = calc_distance(df_rides,
                         col_x1_utm='pickup_x_utm',col_x2_utm='dropoff_x_utm',
                         col_y1_utm='pickup_y_utm',col_y2_utm="dropoff_y_utm",
                         type_='manhattan')

assert(np.all(df_rides.distance_beeline<=df_rides.distance_manhattan))
df_rides.head()

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,pickup_geom,dropoff_geom,pickup_x_utm,pickup_y_utm,dropoff_x_utm,dropoff_y_utm,distance_beeline,distance_manhattan
id,,,,,,,,,,,,,,,,,,
id0161319,2,2016-01-19 20:31:29,2016-01-19 20:39:14,1,-73.993294,40.729328,-74.008316,40.716763,N,465,POINT (-73.99329376220702 40.72932815551758),POINT (-74.00831604003906 40.71676254272461),585011.603343,4.509198e+06,583758.775727,4.507789e+06,1885.659587,2662.130650
id2074074,2,2016-02-03 11:52:45,2016-02-03 12:03:18,2,-73.980499,40.780331,-73.980919,40.765862,N,633,POINT (-73.98049926757811 40.78033065795898),POINT (-73.98091888427734 40.76586151123047),586026.281871,4.514872e+06,586009.532571,4.513265e+06,1606.685826,1623.347819
id2653850,2,2016-05-18 20:58:37,2016-05-18 21:04:50,1,-73.980728,40.738369,-73.993721,40.738682,N,373,POINT (-73.98072814941406 40.73836898803711),POINT (-73.99372100830078 40.73868179321289),586061.075414,4.510214e+06,584963.622848,4.510236e+06,1097.674400,1119.519585
id0836600,2,2016-05-20 15:59:37,2016-05-20 16:07:38,1,-73.982162,40.774487,-73.982979,40.781868,N,481,POINT (-73.98216247558594 40.77448654174805),POINT (-73.98297882080078 40.78186798095703),585893.464665,4.514222e+06,585815.073138,4.515040e+06,822.343308,896.989894
id0184401,1,2016-06-05 12:53:33,2016-06-05 13:21:22,1,-73.992538,40.751705,-73.975433,40.717648,N,1669,POINT (-73.99253845214845 40.75170516967773),POINT (-73.97543334960938 40.71764755249024),585046.878265,4.511683e+06,586534.983744,4.507919e+06,4047.411340,5252.023735


In [14]:
df_rides['avg_speed_kmh'] = (df_rides.distance_manhattan*1e-3)/ (df_rides.trip_duration*1/3600)

In [15]:
from pandas.tseries.holiday import USFederalHolidayCalendar
cal = USFederalHolidayCalendar()

def calc_time_features(df,index_col='pickup_datetime',flg_days_before_after=False): 
    ind = df.set_index(index_col,drop='False').index
    holidays = pd.DataFrame(cal.holidays(start=str(ind.year.min()), end=str(ind.year.max()+1),return_name=True)).rename(columns={0:"holiday_name"})
    if flg_days_before_after: 
        holidays = pd.concat([holidays,
                             "Day Before " + holidays.shift(-1, 'D'),
                             "Day After " + holidays.shift(1, 'D')])
    #holidays.name = 'holiday_name'  # required for join
    df['day_of_week'] = ind.dayofweek 
    df['season'] = (ind.month % 12 + 3) // 3
    df['season'] = ind.quarter
    df['month'] = ind.month
    df['day_of_year'] = ind.dayofyear
    df['is_weekend'] = ind.dayofweek>5
    df['hour'] = ind.hour
    df['date'] = pd.DatetimeIndex(ind.date)
    df['id'] = df.index
    df = df.set_index("date",drop=False).join(holidays,how='left').set_index("id",drop=False)
    df['is_holiday'] = 0 
    df['is_holiday'] = ~(pd.isna(df['holiday_name']))
    return df

df_rides = calc_time_features(df_rides)

In [16]:
df_rides.holiday_name.value_counts()

Presidents Day                6238
New Years Day                 6103
Dr. Martin Luther King Jr.    6092
MemorialDay                   4751
Name: holiday_name, dtype: int64

In [17]:
#To-Do: Compare numba to numpy 
def calc_hours_daylight(df,col_latitude='pickup_latitude',col_day_of_year = "day_of_year"):
    P = np.arcsin(0.39795 * np.cos(0.2163108 + 2 * np.arctan(0.9671396 * np.tan(.00860 * (df[col_day_of_year].astype(int)-186)))))
    hours_daylight = 24 - (24 / np.pi) * np.arccos(
        (np.sin((0.8333 * np.pi / 180) + np.sin(df[col_latitude] * np.pi / 180) * np.sin(P)) / (np.cos(df[col_latitude] * np.pi / 180) * np.cos(P))))
    return df.assign(**{"hours_daylight": hours_daylight})

df_rides = calc_hours_daylight(df_rides)

In [18]:
def calc_cell_id(df,col_x_utm,col_y_utm,col_id='Cell_ID',cell_length=100,keep_coordinates_center=True): 
    df[f"x_sw_utm_{col_id}"] = ((df[col_x_utm].values//cell_length)*cell_length).astype(int)
    df[f"y_sw_utm_{col_id}"] = ((df[col_y_utm].values//cell_length)*cell_length).astype(int)
    df[col_id] = f"{cell_length}mN"+(df[f"x_sw_utm_{col_id}"]//cell_length).astype(str)+"E"+(df[f"y_sw_utm_{col_id}"]//cell_length).astype(str)
    if not keep_coordinates_center: 
        df.drop(columns=[f"x_sw_utm_{col_id}",f"y_sw_utm_{col_id}"],inplace=True)
    return df

df_rides = calc_cell_id(df_rides,col_x_utm="dropoff_x_utm",col_y_utm="dropoff_y_utm",col_id='Cell_ID_dropoff')
df_rides = calc_cell_id(df_rides,col_x_utm="pickup_x_utm",col_y_utm="pickup_y_utm",col_id='Cell_ID_pickup')
df_rides.head()

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration,...,id,holiday_name,is_holiday,hours_daylight,x_sw_utm_Cell_ID_dropoff,y_sw_utm_Cell_ID_dropoff,Cell_ID_dropoff,x_sw_utm_Cell_ID_pickup,y_sw_utm_Cell_ID_pickup,Cell_ID_pickup
id,,,,,,,,,,,,,,,,,,,,,
id0856026,1,2016-01-01 10:26:13,2016-01-01 10:32:34,1,-73.986145,40.746346,-73.999153,40.720112,N,381,...,id0856026,New Years Day,True,9.327102,584500,4508100,100mN5845E45081,585500,4511000,100mN5855E45110
id0535487,1,2016-01-01 02:10:49,2016-01-01 02:21:59,2,-73.985474,40.727470,-73.960594,40.761784,N,670,...,id0535487,New Years Day,True,9.329001,587700,4512800,100mN5877E45128,585600,4508900,100mN5856E45089
id1588995,2,2016-01-01 00:35:52,2016-01-01 00:52:42,1,-74.002487,40.739632,-73.977242,40.750751,N,1010,...,id1588995,New Years Day,True,9.327778,586300,4511500,100mN5863E45115,584200,4510300,100mN5842E45103
id2422054,1,2016-01-01 03:50:40,2016-01-01 04:06:04,2,-73.988609,40.727287,-73.962807,40.712269,N,924,...,id2422054,New Years Day,True,9.329019,587600,4507300,100mN5876E45073,585400,4508900,100mN5854E45089
id2651089,1,2016-01-01 02:45:20,2016-01-01 02:48:57,2,-73.953659,40.771206,-73.953117,40.780071,N,217,...,id2651089,New Years Day,True,9.324600,588300,4514800,100mN5883E45148,588300,4513800,100mN5883E45138


In [19]:
def aggregate_by_ID(df,col_groupby,agg_funcs={'passenger_count':['sum','mean'],'trip_duration':['sum','mean']}):
    agg_funcs = {**agg_funcs,**{f"x_sw_utm_{col_groupby}":'first',f"y_sw_utm_{col_groupby}":'first'}}
    df = df.groupby(col_groupby).agg(agg_funcs)
    df.columns = [f"{agg_func}_{col}" for col,agg_func 
                  in zip(df.columns.get_level_values(level=0),df.columns.get_level_values(level=1))]
    df.rename(columns={c:c.replace("first_","") for c in df.columns},inplace=True)
    return df

In [20]:
df_cells = (aggregate_by_ID(df_rides,
                            col_groupby="Cell_ID_pickup")
            .join(aggregate_by_ID(df_rides,
                                  col_groupby='Cell_ID_dropoff'),
                  how='outer',lsuffix='_pickup',rsuffix='_dropoff')
           )
df_cells.head()

,sum_passenger_count_pickup,mean_passenger_count_pickup,sum_trip_duration_pickup,mean_trip_duration_pickup,x_sw_utm_Cell_ID_pickup,y_sw_utm_Cell_ID_pickup,sum_passenger_count_dropoff,mean_passenger_count_dropoff,sum_trip_duration_dropoff,mean_trip_duration_dropoff,x_sw_utm_Cell_ID_dropoff,y_sw_utm_Cell_ID_dropoff
100mN-37336E53461,4.0,2.0,1604.0,802.0,-3733600.0,5346100.0,4.0,2.0,1604.0,802.0,-3733600.0,5346100.0
100mN10602E40150,1.0,1.0,385.0,385.0,1060200.0,4015000.0,1.0,1.0,385.0,385.0,1060200.0,4015000.0
100mN11396E49446,1.0,1.0,1131.0,1131.0,1139600.0,4944600.0,NaN,NaN,NaN,NaN,NaN,NaN
100mN1210E45194,2.0,2.0,445.0,445.0,121000.0,4519400.0,2.0,2.0,445.0,445.0,121000.0,4519400.0
100mN12837E44073,1.0,1.0,329.0,329.0,1283700.0,4407300.0,1.0,1.0,329.0,329.0,1283700.0,4407300.0


In [21]:
df_cells.to_pickle(os.path.join("../data","df_cells.pickle"))

In [22]:
df_rides.to_pickle(os.path.join("../data","df_rides.pickle"))